# ----------------------------------------------Classification------------------------------------------

Import the library needed

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.models import Sequential
from keras.optimizers import Adam
from keras.models import model_from_json
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from urad_Preprocessing import get_data_preprocessing
np.random.seed(5) #set pseudo checkpoint

Using TensorFlow backend.


# Function

There are several functions in it:
- convert_cpc : to convert static data into time series data
- divide_data : to divide the data into training and test data
- Neural network lists: contains the models of 3 neural network methods

Convert the CPC (for univariate) or multiple features (for multivariate) into time series data

In [2]:
def convert_cpc(dataset, time_prediction=7):
    # look back default value is in day, it is equal to one week
    dataX, dataY = [], []
    for i in range(len(dataset)-time_prediction-1):
        a = dataset[i:(i+time_prediction), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_prediction, 0])
    return np.array(dataX), np.array(dataY)

Divide the data into training, and test data

In [3]:
def divide_data(dataset, time_prediction):
    len_dataset = len(dataset)
    datasetX, datasetY= convert_cpc(dataset, time_prediction)

    #prepare the dataset, and divide them into training and testing dataset
    train_size = int(len_dataset * 0.8)
    test_size = len_dataset - train_size
    trainX, trainY, testX, testY =  datasetX[:train_size,:], datasetY[:train_size], \
                                    datasetX[train_size:,:], datasetY[train_size:]
    return trainX, trainY, testX, testY

Neural network / Deep learning Lists

In [4]:
def build_ANN(learning_rate, hidden_layer, time_prediction):
    print ("---------------Get into ANN function")
    forecast_model = Sequential()
    forecast_model.add(Dense(units = time_prediction*(2**(hidden_layer+1)), kernel_initializer = "uniform", 
                             activation = "relu", input_dim = time_prediction))
    forecast_model.add(Dropout(0.5))
    for i in range(hidden_layer - 1):
        forecast_model.add(Dense(units = time_prediction*(2**(hidden_layer-i)), 
                                 kernel_initializer = "uniform",
                                 activation = "relu"))
        forecast_model.add(Dropout(0.5))
    forecast_model.add(Dense(units = 1, kernel_initializer = "uniform", activation='sigmoid'))
    adam = Adam(decay = learning_rate*0.05, lr=learning_rate)
    forecast_model.compile(loss="binary_crossentropy", optimizer = adam, metrics=['accuracy'])
    return forecast_model

In [5]:
def build_LSTM(learning_rate, hidden_layer, time_prediction):
    print ("\n---------------Get into LSTM function--------------------")  
    print ("Learning rate and hidden layer: ", learning_rate, hidden_layer)
    lstm_model = Sequential()
    #print (learning_rate, hidden_layer, time_prediction)
    lstm_model.add( LSTM(time_prediction*(2**(hidden_layer+1)),
                         input_shape = (1, time_prediction), return_sequences = True)) 
    lstm_model.add(Dropout(0.5))
    for i in range(hidden_layer - 2):
        lstm_model.add( LSTM(    time_prediction*(2**(hidden_layer-i)), 
                                 return_sequences = True ) )
        lstm_model.add(Dropout(0.5))
    
    lstm_model.add( LSTM(time_prediction) )
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units = 1, activation='sigmoid'))
    adam = Adam(decay = learning_rate*0.05, lr=learning_rate)
    lstm_model.compile(loss='binary_crossentropy', optimizer = adam, metrics=['accuracy'])
    return lstm_model

In [8]:
def build_model(learning_rate, method = "ANN", hidden_layer = 1, time_prediction=7):
    model = None
    if method =="ANN":
        model = build_ANN(learning_rate, hidden_layer, time_prediction)
    elif method == "LSTM":
        model = build_LSTM(learning_rate, hidden_layer, time_prediction)
    return model

# Fine Tuning

In [9]:
time_prediction = 10
get_data_preprocessing() #preprocess the raw data and get the important features
dataset = pd.read_csv("fb_ads_preprocess.csv", header=None)
print (dataset.iloc[:5,:])

dataset = dataset.iloc[:, -1].values.reshape(-1, 1)
#print (dataset.shape, dataset[0:5])
trainX, trainY, testX, testY = divide_data(dataset, time_prediction)

#run this code for LSTM
#trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
#testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
#print (trainX.shape, trainY.shape, testX.shape, testY.shape)

Files from the folder-fb ads/ that contains-22 files are succesfully loaded...
Preprocessing is successful!
         0         1    2    3    4    5    6    7    8    9    10   11   12  \
0  0.006113  0.136564  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0   
1  0.032698  0.095485  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   
2  0.042781  0.073221  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0   
3  0.035656  0.087478  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   
4  0.042986  0.098128  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0   

    13  
0  1.0  
1  1.0  
2  1.0  
3  0.0  
4  0.0  


In [10]:
#------------fine tuning ANN using gridSearch
model = KerasRegressor(build_fn=build_model)
parameters = {  'batch_size' : [512, 256, 128], 
                'nb_epoch' : [50000, 10000, 1000],
                'learning_rate': [0.00001, 0.0001, 0.001],
                'hidden_layer' : [3, 2, 1],
                #'optimizer' : ['adagrad', 'rmsprop', 'adam'],
                'time_prediction' :[time_prediction]}

fine_tune = GridSearchCV(estimator=model, param_grid=parameters, cv = 5)
fine_tune_result = fine_tune.fit(trainX, trainY)
best_params = fine_tune_result.best_params_
best_accuracy = fine_tune_result.best_score_

---------------Get into ANN function
Epoch 1/10
8390/8390 [==============================] - 4s - loss: 0.6931 - acc: 0.5076     
Epoch 2/10
8390/8390 [==============================] - 0s - loss: 0.6931 - acc: 0.5063     
Epoch 3/10
8390/8390 [==============================] - 0s - loss: 0.6931 - acc: 0.5011     
Epoch 4/10
8390/8390 [==============================] - 0s - loss: 0.6931 - acc: 0.4982     
Epoch 5/10
8390/8390 [==============================] - 0s - loss: 0.6931 - acc: 0.5036     
Epoch 6/10
8390/8390 [==============================] - 0s - loss: 0.6931 - acc: 0.5008     
Epoch 7/10
8390/8390 [==============================] - 0s - loss: 0.6931 - acc: 0.5073     
Epoch 8/10
8390/8390 [==============================] - 0s - loss: 0.6931 - acc: 0.5069     
Epoch 9/10
8390/8390 [==============================] - 0s - loss: 0.6932 - acc: 0.5037     
Epoch 10/10
 512/8390 [>.............................] - ETA: 0s---------------Get into ANN function
Epoch 1/10
8390/8390 [===

Epoch 1/10
8391/8391 [==============================] - 0s - loss: 0.6932 - acc: 0.4929     
Epoch 2/10
8391/8391 [==============================] - 0s - loss: 0.6932 - acc: 0.4930     
Epoch 3/10
8391/8391 [==============================] - 0s - loss: 0.6932 - acc: 0.4877     
Epoch 4/10
8391/8391 [==============================] - 0s - loss: 0.6932 - acc: 0.4880     
Epoch 5/10
8391/8391 [==============================] - 0s - loss: 0.6932 - acc: 0.4980     
Epoch 6/10
8391/8391 [==============================] - 0s - loss: 0.6932 - acc: 0.4890     
Epoch 7/10
8391/8391 [==============================] - 0s - loss: 0.6932 - acc: 0.4992     
Epoch 8/10
8391/8391 [==============================] - 0s - loss: 0.6932 - acc: 0.4959     
Epoch 9/10
8391/8391 [==============================] - 0s - loss: 0.6932 - acc: 0.4967     
Epoch 10/10
 512/8391 [>.............................] - ETA: 0s---------------Get into ANN function
Epoch 1/10
8391/8391 [==============================] - 0s - l

8390/8390 [==============================] - 0s - loss: 0.6931 - acc: 0.5045     
Epoch 2/10
8390/8390 [==============================] - 0s - loss: 0.6931 - acc: 0.5063     
Epoch 3/10
8390/8390 [==============================] - 0s - loss: 0.6931 - acc: 0.5080     
Epoch 4/10
8390/8390 [==============================] - 0s - loss: 0.6931 - acc: 0.5070     
Epoch 5/10
8390/8390 [==============================] - 0s - loss: 0.6931 - acc: 0.5074     
Epoch 6/10
8390/8390 [==============================] - 0s - loss: 0.6931 - acc: 0.5057     
Epoch 7/10
8390/8390 [==============================] - 0s - loss: 0.6931 - acc: 0.5063     
Epoch 8/10
8390/8390 [==============================] - 0s - loss: 0.6930 - acc: 0.5062     
Epoch 9/10
8390/8390 [==============================] - 0s - loss: 0.6930 - acc: 0.5067     
Epoch 10/10
8390/8390 [==============================] - 0s     
---------------Get into ANN function
Epoch 1/10
8390/8390 [==============================] - 0s - loss: 0.693

8391/8391 [==============================] - 1s - loss: 0.6931 - acc: 0.5046     
Epoch 2/10
8391/8391 [==============================] - 0s - loss: 0.6931 - acc: 0.4989     
Epoch 3/10
8391/8391 [==============================] - 0s - loss: 0.6931 - acc: 0.5013     
Epoch 4/10
8391/8391 [==============================] - 0s - loss: 0.6931 - acc: 0.5152     
Epoch 5/10
8391/8391 [==============================] - 0s - loss: 0.6931 - acc: 0.5173     
Epoch 6/10
8391/8391 [==============================] - 0s - loss: 0.6931 - acc: 0.5159     
Epoch 7/10
8391/8391 [==============================] - 0s - loss: 0.6931 - acc: 0.5200     
Epoch 8/10
8391/8391 [==============================] - 0s - loss: 0.6931 - acc: 0.5239     
Epoch 9/10
8391/8391 [==============================] - 0s - loss: 0.6930 - acc: 0.5320     
Epoch 10/10
 512/8391 [>.............................] - ETA: 0s---------------Get into ANN function
Epoch 1/10
8390/8390 [==============================] - 1s - loss: 0.6931

8390/8390 [==============================] - 1s - loss: 0.6932 - acc: 0.4931     
Epoch 2/10
8390/8390 [==============================] - 0s - loss: 0.6930 - acc: 0.5082     
Epoch 3/10
8390/8390 [==============================] - 0s - loss: 0.6924 - acc: 0.5111     
Epoch 4/10
8390/8390 [==============================] - 0s - loss: 0.6907 - acc: 0.5381     
Epoch 5/10
8390/8390 [==============================] - 0s - loss: 0.6879 - acc: 0.5482     
Epoch 6/10
8390/8390 [==============================] - 0s - loss: 0.6862 - acc: 0.5484     
Epoch 7/10
8390/8390 [==============================] - 0s - loss: 0.6866 - acc: 0.5507     
Epoch 8/10
8390/8390 [==============================] - 0s - loss: 0.6839 - acc: 0.5591     
Epoch 9/10
8390/8390 [==============================] - 0s - loss: 0.6852 - acc: 0.5563     
Epoch 10/10
 512/8390 [>.............................] - ETA: 0s---------------Get into ANN function
Epoch 1/10
8391/8391 [==============================] - 1s - loss: 0.6931

...

8390/8390 [==============================] - 0s - loss: 0.6930 - acc: 0.5114     
Epoch 7/10
8390/8390 [==============================] - 0s - loss: 0.6930 - acc: 0.5054     
Epoch 8/10
8390/8390 [==============================] - 0s - loss: 0.6930 - acc: 0.5133     
Epoch 9/10
8390/8390 [==============================] - 0s - loss: 0.6931 - acc: 0.5089     
Epoch 10/10
7680/8390 [==========================>...] - ETA: 0s---------------Get into ANN function
Epoch 1/10
8390/8390 [==============================] - 7s - loss: 0.6930 - acc: 0.5060     
Epoch 2/10
8390/8390 [==============================] - 0s - loss: 0.6931 - acc: 0.5030     
Epoch 3/10
8390/8390 [==============================] - 0s - loss: 0.6931 - acc: 0.5041     
Epoch 4/10
8390/8390 [==============================] - 0s - loss: 0.6930 - acc: 0.5092     
Epoch 5/10
8390/8390 [==============================] - 0s - loss: 0.6930 - acc: 0.5021     
Epoch 6/10
8390/8390 [==============================] - 0s - loss: 0.6931

8391/8391 [==============================] - 0s - loss: 0.6931 - acc: 0.4971     
Epoch 7/10
8391/8391 [==============================] - 0s - loss: 0.6931 - acc: 0.5084     
Epoch 8/10
8391/8391 [==============================] - 0s - loss: 0.6930 - acc: 0.5083     
Epoch 9/10
8391/8391 [==============================] - 0s - loss: 0.6931 - acc: 0.5015     
Epoch 10/10
8391/8391 [==============================] - 0s     
---------------Get into ANN function
Epoch 1/10
8390/8390 [==============================] - 7s - loss: 0.6931 - acc: 0.5136     
Epoch 2/10
8390/8390 [==============================] - 0s - loss: 0.6930 - acc: 0.5048     
Epoch 3/10
8390/8390 [==============================] - 0s - loss: 0.6930 - acc: 0.5094     
Epoch 4/10
8390/8390 [==============================] - 0s - loss: 0.6928 - acc: 0.5271     
Epoch 5/10
8390/8390 [==============================] - 0s - loss: 0.6926 - acc: 0.5255     
Epoch 6/10
8390/8390 [==============================] - 0s - loss: 0.692

8390/8390 [==============================] - 0s - loss: 0.6925 - acc: 0.5366     
Epoch 6/10
8390/8390 [==============================] - 0s - loss: 0.6923 - acc: 0.5410     
Epoch 7/10
8390/8390 [==============================] - 0s - loss: 0.6921 - acc: 0.5395     
Epoch 8/10
8390/8390 [==============================] - 0s - loss: 0.6919 - acc: 0.5386     
Epoch 9/10
8390/8390 [==============================] - 0s - loss: 0.6918 - acc: 0.5418     
Epoch 10/10
7168/8390 [========================>.....] - ETA: 0s---------------Get into ANN function
Epoch 1/10
8391/8391 [==============================] - 7s - loss: 0.6929 - acc: 0.5209     
Epoch 2/10
8391/8391 [==============================] - 0s - loss: 0.6928 - acc: 0.5150     
Epoch 3/10
8391/8391 [==============================] - 0s - loss: 0.6926 - acc: 0.5260     
Epoch 4/10
8391/8391 [==============================] - 0s - loss: 0.6925 - acc: 0.5223     
Epoch 5/10
8391/8391 [==============================] - 0s - loss: 0.6924

8390/8390 [==============================] - 0s - loss: 0.6861 - acc: 0.5520     
Epoch 6/10
8390/8390 [==============================] - 0s - loss: 0.6857 - acc: 0.5539     
Epoch 7/10
8390/8390 [==============================] - 0s - loss: 0.6851 - acc: 0.5516     
Epoch 8/10
8390/8390 [==============================] - 0s - loss: 0.6843 - acc: 0.5569     
Epoch 9/10
8390/8390 [==============================] - 0s - loss: 0.6846 - acc: 0.5549     
Epoch 10/10
7552/8390 [==========================>...] - ETA: 0s---------------Get into ANN function
Epoch 1/10
8390/8390 [==============================] - 7s - loss: 0.6926 - acc: 0.5157     
Epoch 2/10
8390/8390 [==============================] - 0s - loss: 0.6909 - acc: 0.5485     
Epoch 3/10
8390/8390 [==============================] - 0s - loss: 0.6884 - acc: 0.5522     
Epoch 4/10
8390/8390 [==============================] - 0s - loss: 0.6857 - acc: 0.5541     
Epoch 5/10
8390/8390 [==============================] - 0s - loss: 0.6844

8391/8391 [==============================] - 0s - loss: 0.6846 - acc: 0.5551     
Epoch 6/10
8391/8391 [==============================] - 0s - loss: 0.6844 - acc: 0.5548     
Epoch 7/10
8391/8391 [==============================] - 0s - loss: 0.6837 - acc: 0.5556     
Epoch 8/10
8391/8391 [==============================] - 0s - loss: 0.6836 - acc: 0.5557     
Epoch 9/10
8391/8391 [==============================] - 0s - loss: 0.6841 - acc: 0.5589     
Epoch 10/10
7552/8391 [==========================>...] - ETA: 0s---------------Get into ANN function
Epoch 1/10
8391/8391 [==============================] - 8s - loss: 0.6925 - acc: 0.5246     
Epoch 2/10
8391/8391 [==============================] - 0s - loss: 0.6902 - acc: 0.5458     
Epoch 3/10
8391/8391 [==============================] - 0s - loss: 0.6875 - acc: 0.5459     
Epoch 4/10
8391/8391 [==============================] - 0s - loss: 0.6842 - acc: 0.5550     
Epoch 5/10
8391/8391 [==============================] - 0s - loss: 0.6821

print the loss and the best parameters

In [11]:
means = fine_tune_result.cv_results_['mean_test_score']
stds = fine_tune_result.cv_results_['std_test_score']
params = fine_tune_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("loss: %f std: (%f) with: %r" % (mean, stdev, param))

0.693143 (0.000006) with: {'hidden_layer': 3, 'time_prediction': 10, 'nb_epoch': 50000, 'batch_size': 512, 'learning_rate': 1e-05}
0.693141 (0.000004) with: {'hidden_layer': 3, 'time_prediction': 10, 'nb_epoch': 10000, 'batch_size': 512, 'learning_rate': 1e-05}
0.693139 (0.000006) with: {'hidden_layer': 3, 'time_prediction': 10, 'nb_epoch': 1000, 'batch_size': 512, 'learning_rate': 1e-05}
0.692999 (0.000083) with: {'hidden_layer': 3, 'time_prediction': 10, 'nb_epoch': 50000, 'batch_size': 512, 'learning_rate': 0.0001}
0.692993 (0.000088) with: {'hidden_layer': 3, 'time_prediction': 10, 'nb_epoch': 10000, 'batch_size': 512, 'learning_rate': 0.0001}
0.693001 (0.000066) with: {'hidden_layer': 3, 'time_prediction': 10, 'nb_epoch': 1000, 'batch_size': 512, 'learning_rate': 0.0001}
0.684178 (0.005322) with: {'hidden_layer': 3, 'time_prediction': 10, 'nb_epoch': 50000, 'batch_size': 512, 'learning_rate': 0.001}
0.683963 (0.005411) with: {'hidden_layer': 3, 'time_prediction': 10, 'nb_epoch': 1

In [ ]:
#using the training data that will be divided again into 80% train, and 20% validation
lstm_model = build_ANN(0.01, 1, time_prediction)
batch_size = 256
history = lstm_model.fit(trainX, trainY, epochs=100, batch_size=batch_size, validation_split=0.2, shuffle=True)